# Data Validation

Here we will validate our regridded data with in situ data. We are going to use data from 3 different sources to validate our model: PN BOIA's current data, SIMCosta's current data and SISBIOTA temperature data.

### Required Libraries

In [1]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd
import numpy as np

# netCDF manipulation
from netCDF4 import Dataset

### PN Boia




Relevante info: 
* Project's data: https://www.marinha.mil.br/chm/dados-do-pnboiaboias/boia-itajai
* DMS: Lat 27°24,35'S / Lon 047°15,93'W
* DD: Lat 27.4° / Lon 47.2655°

- cvelX: velocidades das correntes em mm/s na celula X,onde X varia de 1 a 20. Quanto maior o número da célula, mais afastada é a medição em relação ao ADCP;

- cvelXflagid: Identificador de flag da velocidades das correntes na celula X, onde X varia de 1 a 20. Quanto maior o número da célula, mais afastada é a medição em relação ao ADCP;

- cdirX: direcao das correntes na celula X,onde X varia de 1 a 20. Quanto maior o número da célula, mais afastada é a medição em relação ao ADCP; e



In [2]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("/data/validation/pnboia_adcptratados_itajai.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,data,Cvel1,Cvel1flagid,Cdir1,Cdir1flagid,Cvel2,Cvel2flagid,Cdir2,Cdir2flagid,Cvel3,...,Cdir18,Cdir18flagid,Cvel19,Cvel19flagid,Cdir19,Cdir19flagid,Cvel20,Cvel20flagid,Cdir20,Cdir20flagid
0,2011-02-17 12:00:00,80,0,124.0,0,88,0,120.0,0,70,...,65.0,0,776,0,79.0,0,707,0,91.0,0
1,2011-02-17 13:00:00,176,61,187.0,61,188,61,182.0,61,174,...,60.0,10,283,10,79.0,10,511,10,102.0,10
2,2011-02-17 14:00:00,223,0,168.0,0,202,0,179.0,0,202,...,106.0,10,240,10,147.0,10,442,10,152.0,10
3,2011-02-17 15:00:00,203,0,186.0,0,210,0,191.0,0,208,...,123.0,10,235,10,98.0,10,167,10,130.0,10
4,2011-02-17 16:00:00,218,0,211.0,0,218,0,218.0,0,207,...,149.0,0,470,0,114.0,0,409,0,142.0,0


We will need the data for the first depth and convert bearing and intensity to u v components

In [36]:
df = data[['data', 'Cdir1', 'Cvel1']].copy()

# Clean nan values
df.drop(df[df.Cdir1 == -9999].index, inplace=True)
df.drop(df[df.Cvel1 == -9.999].index, inplace=True)

# Convert velocity from mm/s to m/s
df['Cvel1'] = df['Cvel1']/1000


df['theta'] = np.nan
# Create u v components for the first quadrant
search = (df.Cdir1 <= 90)
df.loc[search, 'theta'] = 90 - df.Cdir1
df.loc[search, 'u0'] = np.cos(df['theta']) * df['Cvel1']
df.loc[search, 'v0'] = np.sin(df['theta']) * df['Cvel1']

# Create u v components for the second quadrant
search = (df.Cdir1 <= 180) & (df.Cdir1 > 90)
df.loc[search, 'theta'] = df.Cdir1 - 90
df.loc[search, 'u0'] = np.cos(df['theta']) * df['Cvel1']
df.loc[search, 'v0'] = np.sin(df['theta']) * df['Cvel1'] * -1

# # # # Create u v components for the third quadrant
search = (df.Cdir1 <= 270) & (df.Cdir1 > 180)
df.loc[search, 'theta'] = 270 - df.Cdir1
df.loc[search, 'u0'] = np.cos(df['theta']) * df['Cvel1'] * -1
df.loc[search, 'v0'] = np.sin(df['theta']) * df['Cvel1'] * -1

# # # # Create u v components for the fourth quadrant
search = (df.Cdir1 <= 360) & (df.Cdir1 > 270)
df.loc[search, 'theta'] = df.Cdir1 - 270
df.loc[search, 'u0'] = np.cos(df['theta']) * df['Cvel1'] * -1
df.loc[search, 'v0'] = np.sin(df['theta']) * df['Cvel1']

df.head()

,data,Cdir1,Cvel1,theta,u0,v0
0,2011-02-17 12:00:00,124.0,0.080,34.0,-0.067886,-0.042327
1,2011-02-17 13:00:00,187.0,0.176,83.0,-0.043919,-0.170432
2,2011-02-17 14:00:00,168.0,0.223,78.0,-0.191290,-0.114617
3,2011-02-17 15:00:00,186.0,0.203,84.0,0.138045,-0.148838
4,2011-02-17 16:00:00,211.0,0.218,59.0,0.168095,-0.138809


In [44]:
df.describe()

,Cdir1,Cvel1,theta,u0,v0
count,26429.000000,26429.000000,26429.000000,26429.000000,26429.000000
mean,189.666011,0.241711,53.079307,-0.003195,-0.004240
std,78.103161,0.156577,25.533307,0.203900,0.203322
min,0.000000,0.001000,0.000000,-1.013908,-1.049552
25%,151.000000,0.132000,33.000000,-0.122772,-0.122652
50%,190.000000,0.204000,58.000000,-0.001991,-0.002337
75%,236.000000,0.312000,75.000000,0.118983,0.116329
max,360.000000,1.174000,90.000000,1.058627,0.963315


### Regrided data

The data from Global reanalysis have different velocity fields

```
u > CF Field: eastward_sea_water_velocity(time(724), depth(5), latitude(7), longitude(8)) m s-1
v > CF Field: northward_sea_water_velocity(time(724), depth(5), latitude(7), longitude(8)) m s-1
```

In [40]:
nc_file = '/data/validation/grid_500m_v0-validation.nc'
nc_fid = Dataset(nc_file, 'r')  # Dataset is the class behavior to open the file
                                # and create an instance of the ncCDF4 class
time = nc_fid.variables['time'][:]
lats = nc_fid.variables['latitude'][:]  # extract/copy the data
lons = nc_fid.variables['longitude'][:]

In [41]:
nc_fid

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.8
    julian_day_unit: days since 1950-01-01 00:00:00
    history: 2019/04/04 11:59:50 MERCATOR OCEAN Netcdf creation
    forecast_type: hindcast
    bulletin_date: 2018-12-26 00:00:00
    references: http://www.mercator-ocean.fr
    _CoordSysBuilder: ucar.nc2.dataset.conv.CF1Convention
    easting: longitude
    domain_name: GL12
    FROM_ORIGINAL_FILE__latitude_min: -80.0
    northing: latitude
    z_max: 5727.917
    forecast_range: 6-day_forecast
    institution: MERCATOR OCEAN
    source: MERCATOR GLORYS12V1
    FROM_ORIGINAL_FILE__longitude_min: -180.0
    title: daily mean fields from Global Ocean Physics Analysis and Forecast updated Daily
    field_date: 2018-12-25 00:00:00
    FROM_ORIGINAL_FILE__field_type: mean
    FROM_ORIGINAL_FILE__latitude_max: 90.0
    comment: CMEMS product
    field_julian_date: 25195.0
    bulletin_type: operational
    FROM_ORIGINAL_FILE__lon

In [45]:
lats 27.4° / Lon 47.2655

masked_array(data=[-27.66666   , -27.66204889, -27.65743778, -27.65282667,
                   -27.64821556, -27.64360444, -27.63899333, -27.63438222,
                   -27.62977111, -27.62516   , -27.62054889, -27.61593778,
                   -27.61132667, -27.60671556, -27.60210444, -27.59749333,
                   -27.59288222, -27.58827111, -27.58366   , -27.57904889,
                   -27.57443778, -27.56982667, -27.56521556, -27.56060444,
                   -27.55599333, -27.55138222, -27.54677111, -27.54216   ,
                   -27.53754889, -27.53293778, -27.52832667, -27.52371556,
                   -27.51910444, -27.51449333, -27.50988222, -27.50527111,
                   -27.50066   , -27.49604889, -27.49143778, -27.48682667,
                   -27.48221556, -27.47760444, -27.47299333, -27.46838222,
                   -27.46377111, -27.45916   , -27.45454889, -27.44993778,
                   -27.44532667, -27.44071556, -27.43610444, -27.43149333,
                   -27.42

In [46]:
lons

masked_array(data=[-47.5       , -47.49538889, -47.49077778, -47.48616667,
                   -47.48155556, -47.47694444, -47.47233333, -47.46772222,
                   -47.46311111, -47.4585    , -47.45388889, -47.44927778,
                   -47.44466667, -47.44005556, -47.43544444, -47.43083333,
                   -47.42622222, -47.42161111, -47.417     , -47.41238889,
                   -47.40777778, -47.40316667, -47.39855556, -47.39394444,
                   -47.38933333, -47.38472222, -47.38011111, -47.3755    ,
                   -47.37088889, -47.36627778, -47.36166667, -47.35705556,
                   -47.35244444, -47.34783333, -47.34322222, -47.33861111,
                   -47.334     , -47.32938889, -47.32477778, -47.32016667,
                   -47.31555556, -47.31094444, -47.30633333, -47.30172222,
                   -47.29711111, -47.2925    , -47.28788889, -47.28327778,
                   -47.27866667, -47.27405556, -47.26944444, -47.26483333,
                   -47.26